## Task1

-> Objetivo: Dada uma consulta especificada pelo usuário e uma lista de produtos correspondentes, o objetivo desta tarefa é classificar os produtos para que os produtos relevantes sejam classificados acima dos não relevantes.

A entrada para esta tarefa será uma lista de consultas com seus identificadores. O sistema terá que gerar um arquivo CSV onde o **query_id** estará na primeira coluna e o **product_id** na segunda coluna, onde para cada **query_id** primeira linha será o produto mais relevante e a última linha o produto menos relevante

Proximos Passos 

1 Criar seçao de metricas

2 criar modelo 

In [52]:
!pip install lightgbm
!pip install annoy
!pip install faiss-cpu
!pip install -U sentence-transformers
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [53]:
#imports 
import pandas as pd
import numpy as np
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder 
#metricas
from sklearn.metrics import ndcg_score

#modelos
import lightgbm as lgb
import faiss
from annoy import AnnoyIndex

from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import requests
from io import StringIO

from sentence_transformers import SentenceTransformer
import faiss
import math
import matplotlib.pylab as plt




In [54]:
train_df = pd.read_csv('dataset/train-v0.3.csv')
product = pd.read_csv('dataset/product_catalogue-v0.3.csv')
teste_df = pd.read_csv('dataset/test_public-v0.3.csv')

In [55]:
print("shape train data = " , train_df.shape)
print("shape train product = " , product.shape)
print("shape teste product = " , teste_df.shape)



shape train data =  (781744, 5)
shape train product =  (883868, 7)
shape teste product =  (48696, 4)


In [56]:
full_train = pd.merge(train_df, 
                          product[["product_id", "product_locale", "product_title"]],
                          left_on=["product_id", "query_locale"], 
                          right_on=["product_id", "product_locale"]
)
print("shape train product = " , full_train.shape)

shape train product =  (781744, 7)


In [57]:
full_train['id']=full_train.index

In [58]:
full_train = full_train.dropna()
print("shape train product = " , full_train.shape)

shape train product =  (781644, 8)


In [59]:
full_train.isnull().sum()

query_id          0
query             0
query_locale      0
product_id        0
esci_label        0
product_locale    0
product_title     0
id                0
dtype: int64

In [60]:
#full_train.info()
#full_train.head()

**Removendo StopWords / lowcase / caracteres especiais** 

In [61]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [62]:
mask = (full_train['product_locale'] == 'us')
full_train = full_train[mask]
full_train = full_train.iloc[:500,:]
#full_train

In [63]:
validation = full_train.iloc[500:1000,:]
validation

,query_id,query,query_locale,product_id,esci_label,product_locale,product_title,id


In [64]:
text = full_train['product_title']


In [65]:
#tokenizando text
#text_tokens = word_tokenize(text)
stop_words = stopwords.words('english')

text= text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [66]:
text = text.str.lower()
#revome caracteres especiais 
text = text.str.replace('[!,#,@,&,%,0-9]','')


/tmp/ipykernel_9838/4110833427.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('[!,#,@,&,%,0-9]','')


**Criando vetores**

In [67]:
import requests
from io import StringIO
#import pandas as pd

In [68]:
#sentences = train_df['query'].tolist()
sentences = text.tolist()
len(sentences)


500

In [69]:
# remove duplicates and NaN
#sentences = [word for word in list(set(sentences)) if type(word) is str]
#sentences = [word for word in list(set(sentences)) if type(word) is str]
len(sentences)

500

In [70]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens') 
#paraphrase-multilingual-MiniLM-L12-v2

In [71]:
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
faiss.normalize_L2(sentence_embeddings) ## Normalising the Embeddings
sentence_embeddings.shape

(500, 768)

Tratamento de dados de teste

## IndexFlatL2



In [72]:
import faiss

In [73]:
size = sentence_embeddings.shape[1]
size

768

In [74]:
index = faiss.IndexFlatL2(size)
index.is_trained
index.add(sentence_embeddings)

In [75]:
index.ntotal

500

In [76]:
## We get a 768 dimension vector using Roberta. So we will create FAISS index with dimaensions - 768

dim=size
ncentroids=50 ## This is a hyperparameter, and indicates number of clusters to be split into
m=16 ## This is also a hyper parameter
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(sentence_embeddings) ## This step, will do the clustering and create the clusters
print(index.is_trained)
faiss.write_index(index, "trained.index")

WARNING clustering 500 points to 50 centroids: please provide at least 1950 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least

True


In [77]:
### We have to add the embeddings to the Trained Index.
ids=full_train['id'].tolist()

ids=np.array(ids)
index.add_with_ids(sentence_embeddings,ids)
print(index.ntotal)


500


In [78]:
#Teste
k = 4
xq = model.encode(["envelopes without security tint"])

In [79]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[485 486 484 477]]
CPU times: user 0 ns, sys: 1.81 ms, total: 1.81 ms
Wall time: 1.14 ms


In [80]:
import math

In [81]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [82]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    ## Convert the query into embeddings
    query_embedding = model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    #faiss.normalize_L2(query_embedding)
  
    
    index.nprobe=nprobe
    
    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    print(search_result)
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [83]:
query="small price tags with string"
search_result=searchFAISSIndex(full_train,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','product_id','product_title','esci_label','cosine_sim','L2_score']]

     id    cosine_sim    L2_score
0   468 -30884.525610  248.537827
1   473 -31706.057662  251.821594
2   497 -31937.215280  252.737869
3    31 -32128.078806  253.491928
4    32 -32128.078806  253.491928
5    30 -32128.078806  253.491928
6   496 -32186.449902  253.722092
7   467 -32249.520270  253.970551
8   466 -32249.520270  253.970551
9     5 -32297.556776  254.159622
10    6 -32297.556776  254.159622
11   17 -32309.894407  254.208160
12   16 -32309.894407  254.208160
13   15 -32309.894407  254.208160
14  499 -32330.296808  254.288406
15  475 -32368.356588  254.438034
16   12 -32382.311495  254.492874
17   10 -32382.311495  254.492874
18   14 -32382.311495  254.492874
19   13 -32382.311495  254.492874


In [84]:
search_result

,id,product_id,product_title,esci_label,cosine_sim,L2_score
14,468,B001E6C3AE,"Avery 12201 Medium-Weight White Marking Tags, ...",exact,-30884.525610,248.537827
15,473,B00I0EU22Y,"Merchandise Tags #8, White, Scalloped, Hole (n...",exact,-31706.057662,251.821594
18,497,B0772W8DPP,"YEJI 100 pcs Kraft Paper Tags, Gift Tags with ...",irrelevant,-31937.215280,252.737869
10,31,B00OFNI9VK,"Ticonderoga Wood-Cased Pencils, #2 HB Soft, Pr...",substitute,-32128.078806,253.491928
11,32,B00OFNI9VK,"Ticonderoga Wood-Cased Pencils, #2 HB Soft, Pr...",exact,-32128.078806,253.491928
9,30,B00OFNI9VK,"Ticonderoga Wood-Cased Pencils, #2 HB Soft, Pr...",exact,-32128.078806,253.491928
17,496,B076BBJYRN,"White Paper Tags, Jewelry Price Tags with stri...",substitute,-32186.449902,253.722092
13,467,B001E682C2,"AVERY White Marking Tags Strung, Pack of 1000 ...",substitute,-32249.520270,253.970551
12,466,B001E682C2,"AVERY White Marking Tags Strung, Pack of 1000 ...",substitute,-32249.520270,253.970551
1,6,B0002LCZV4,"TICONDEROGA Tri-Write Triangular Pencils, Stan...",exact,-32297.556776,254.159622


In [85]:
#train_df['query'].iloc[[19805, 4829, 978, 14630]]
#10620 14711  1269 15458
sentences[485]

'kc store fixtures   merchandise tag without string /" x -/" white (pack )'

In [86]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, size))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

RuntimeError: Error in faiss::DirectMap::idx_t faiss::DirectMap::get(faiss::DirectMap::idx_t) const at /project/faiss/faiss/invlists/DirectMap.cpp:81: direct map not initialized

In [ ]:
vecs.shape

(4, 768)

In [ ]:
vecs[0][:100]

array([ 4.62268680e-01,  7.02288866e-01,  1.60046089e+00, -2.52886891e-01,
        6.94889724e-02,  3.85936290e-01,  1.86582851e+00,  1.95366383e-01,
       -1.22308277e-01,  4.64537621e-01, -1.35619247e+00, -5.33786297e-01,
        5.25341094e-01,  6.70681655e-01,  1.05484235e+00,  3.25322360e-01,
       -7.39459693e-01, -2.09946677e-01,  1.82559446e-01, -2.91459441e-01,
       -6.66369274e-02,  2.44928196e-01,  3.32280397e-01, -1.15091193e+00,
       -2.93901235e-01, -1.78813443e-01,  2.95608163e-01, -3.64545673e-01,
       -1.53809154e+00,  2.25750789e-01,  1.12063624e-01,  4.58471775e-01,
        6.88506067e-01,  3.36156100e-01,  1.23908556e+00, -1.64252013e-01,
        8.09108242e-02, -1.37895525e+00, -1.68415979e-01, -6.49951220e-01,
       -5.58319427e-02,  6.10021830e-01,  6.05048060e-01,  3.45182210e-01,
        3.01262170e-01, -1.40101707e-03,  1.93336523e+00, -1.23036504e-01,
        3.83745097e-02, -7.41975367e-01, -9.59856808e-02, -4.04200792e-01,
        1.97196960e+00,  

# implementando um indice plano faiss

In [ ]:
index_x = faiss.IndexFlatIP(size)

In [ ]:
index_x.add(sentence_embeddings)

In [ ]:
xq = model.encode(["car paint", "arsoft"])

In [ ]:
#Teste
k = 10
D, I = index_x.search(xq, k)
print(I)

[[289056 122675 335018 312637 103322  42819 195218 141480 226699 236514]
 [ 26941 285127  74533 304948 181771 158355 232285 171911  30746 136225]]


In [ ]:
#train_df['query'].iloc[[10365, 12349, 7469, 7340]]
#12516 15213  7553 12372 18626 15555 11820 20000  5935  4487
sentences[285127]

'over under'

In [ ]:
nbits = size*4  # resolution of bucketed vectors
# initialize index and add vectors
index_e = faiss.IndexLSH(size, nbits)
index_e.add(sentence_embeddings)
# and search
D, I = index_e.search(xq, k)
print(I)

NameError: name 'size' is not defined

In [ ]:
#train_df['query'].iloc[[12349, 7469, 9072, 10365, 4282, 2151, 7340, 50, 11175, 2665 ]]
#[15213  7553  2488 12516  4900 11820 12372 15555 18626  5935
sentences[4900]

'kwikset 00-0 signature series deadbolt pack of  satin nickel'

In [ ]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index_h = faiss.IndexHNSWFlat(size, M)
# set efConstruction and efSearch parameters
index_h.hnsw.efConstruction = ef_construction
index_h.hnsw.efSearch = ef_search
# add data to index
index_h.add(xq)

# search as usual
D, I = index.search(xq, k)
print(I)

[[117613  49427 218312 285342 324777  98247 209897 174188  43960 171233]
 [175480 135044 291961  59109 342169 110316  36749 195076  81267 231094]]


In [ ]:
sentences[20000]

'dellytop womens casual tank tops button up v neck plain sleeveless shirts summer tees army green'